In [2]:
import numpy as np
import os 
import pandas as pd


Valores importantes

K = numero de clusters
p = numero de matrizes de dissimilaridade
n = numero de exemplos


Dimensionalidades:
G = p x K  
U = N x K  
A = p x K
D = p x n x n
D' = n x k 

term_1  = K x n
term_2  = x 

In [6]:
%%time
mode = 'no_class'
metrics = ['euclidean','cityblock','chebyshev']

input_path = [
    os.path.join( '../data/diss_matrices/{}/diss_matrix_{}.csv'.format(mode,metric) ) for metric in metrics
    ]
print(input_path)
D = [np.expand_dims(pd.read_csv(filename).to_numpy(), axis=0) for filename in input_path]

D = np.concatenate( D,axis=0)
D = D[:,:,1:]

['../data/diss_matrices/no_class/diss_matrix_euclidean.csv', '../data/diss_matrices/no_class/diss_matrix_cityblock.csv', '../data/diss_matrices/no_class/diss_matrix_chebyshev.csv']
CPU times: user 8min 29s, sys: 21.5 s, total: 8min 51s
Wall time: 10min 21s


In [26]:
D.shape

(3, 20866, 20866)

In [5]:
!pip install pytables

ERROR: Could not find a version that satisfies the requirement pytables (from versions: none)
ERROR: No matching distribution found for pytables


In [4]:
%%time
store = pd.HDFStore("diss_matrices.h5")
store['euclidean'] = pd.read_csv('../data/diss_matrices/no_class/diss_matrix_euclidean.csv')
store['cityblock'] = pd.read_csv('../data/diss_matrices/no_class/diss_matrix_cityblock.csv')
store['chebyshev'] = pd.read_csv('../data/diss_matrices/no_class/diss_matrix_chebyshev.csv')
store.close()

ImportError: Missing optional dependency 'pytables'.  Use pip or conda to install pytables.

In [ ]:
store = pd.HDFStore("diss_matrices.h5")

ImportError: Missing optional dependency 'pytables'.  Use pip or conda to install pytables.

In [ ]:
%%time
D_ = [np.expand_dims(store[matrix].to_numpy(), axis=0) for matrix in store]
D_ = np.concatenate(D_,axis=0)
D_ = D_[:,:,1:]

CPU times: user 3.79 s, sys: 8.5 s, total: 12.3 s
Wall time: 18.7 s


In [ ]:
D_.shape

(3, 20866, 20866)

In [ ]:
store.close()

In [ ]:
def normalize_data(D):
    print(D.shape)
    l = ([p.sum(axis=1).argmin() for p in list(D)])
    print(np.asarray(l).shape)
    k  =[ np.sum(D[i][l[i]]) for i in range(len(l)) ]
    k = np.asarray(k)[...,np.newaxis,np.newaxis]
    D = np.divide(D,k)
    return D
    
    
l = normalize_data(D)
print(l)

(3, 20866, 20866)
(3,)
[[[0.00000000e+00 9.04564658e-05 4.81121897e-05 ... 8.94753757e-05
   8.89848307e-05 9.90900371e-05]
  [9.04564658e-05 0.00000000e+00 9.04564658e-05 ... 2.97837269e-05
   3.78023979e-05 5.26942604e-05]
  [4.81121897e-05 9.04564658e-05 0.00000000e+00 ... 8.94753757e-05
   8.89848307e-05 9.90900371e-05]
  ...
  [8.94753757e-05 2.97837269e-05 8.94753757e-05 ... 0.00000000e+00
   6.75861302e-05 8.24779927e-05]
  [8.89848307e-05 3.78023979e-05 8.89848307e-05 ... 6.75861302e-05
   0.00000000e+00 1.81719879e-05]
  [9.90900371e-05 5.26942604e-05 9.90900371e-05 ... 8.24779927e-05
   1.81719879e-05 0.00000000e+00]]

 [[0.00000000e+00 7.99588743e-05 5.41386398e-05 ... 6.62175298e-05
   7.96542081e-05 1.04486870e-04]
  [7.99588743e-05 0.00000000e+00 8.35635583e-05 ... 3.27446614e-05
   4.42273267e-05 5.13408268e-05]
  [5.41386398e-05 8.35635583e-05 0.00000000e+00 ... 1.00215293e-04
   6.89144872e-05 9.07694484e-05]
  ...
  [6.62175298e-05 3.27446614e-05 1.00215293e-04 ... 0.

In [7]:
def select_medoids_dist(D,G):
    return np.stack([D[i,G[i],:] for i in range(D.shape[0])], axis=0 )


In [8]:
def compute_J(G,U,D,A,m=2):
    Dt = select_medoids_dist(D,G)
    Ut = np.power(U,m)
    A = np.einsum('pk,pkn->kn',A,Dt) 
    return np.einsum('kn,kn->',A,Ut)

In [9]:
def compute_U_fuzzy_part(G, U, D, A, m):
    Dt = select_medoids_dist(D,G)
    term_1  = np.einsum('pk,pkn->kn',A,Dt)
    term_2  = np.einsum('pk,pkn->n',A,Dt )  

    l = np.divide(term_1, term_2)

    k = np.power( l, (1/(m-1)))
    k[np.where(k == 0)] = 1
    
    Ut = np.divide(1,k)
    return Ut

In [10]:
def search_best_relev_vec(G,U,D,A,m):
    Dt = select_medoids_dist(D,G)

    Ut = np.power(U,m)
    l = np.einsum('kn,pkn->pk',Ut,Dt)

    a = np.prod(l,axis=0)
    a = np.power(a,(1/(D.shape[0])))
    al = np.divide(a,l)
    
    return al

In [11]:
def search_best_medoids(G,U,D,A,m):

    all = []
    for p in range (D.shape[0]):
        al = []
        for k in range(U.shape[0]):
            h = np.sum(np.power(U[k],m)*D[p],axis=1).argmin()
            al.append(h)
        all.append(al)
    
    all = np.asarray(all)
    return all


In [15]:
def mv_algorithm(D, K, m, T, e):

    """
    
    """

    ########## CONSTRAINTS ##############
     
    assert D.shape[1] == D.shape[2]
    assert K > 1 and K < D.shape[1]
    assert e > 0 
    assert m > 1

    ######### INITIALIZATION #############

    p = D.shape[0]
    n = D.shape[1]
  

    G          = np.random.randint(low=0,high=n,size=(p,K))
    A          = np.ones(shape=(p,K))
    U          = compute_U_fuzzy_part(G,None,D,A,m)

    
   
    cur_u  = compute_J(G,U,D,A,m)
    


    last_u = 0

    ########## TRAINING LOOP ###############
    for t in range(T):

        G = search_best_medoids(G,U,D,A,m)
        A = search_best_relev_vec(G,U,D,A,m)
        

        U          = compute_U_fuzzy_part(G,U,D,A,m)
        


        
        
        last_u = cur_u
        cur_u = compute_J(G,U,D,A,m)
        print("Epoch {} ---- dif = {}".format(t,np.absolute(cur_u-last_u)))
        if np.absolute(cur_u-last_u) < e:
            print('saiu com {}'.format(t))
            break

In [16]:
print(D.shape)
K = 5
T = 10
e = 0.00000000000000001
m = 3

(3, 20866, 20866)


In [17]:
mv_algorithm(D.astype(np.float32), K, m, T, e)

Epoch 0 ---- dif = 91.6691478900058
Epoch 1 ---- dif = 0.22615734171372992
Epoch 2 ---- dif = 0.05445445462811449
Epoch 3 ---- dif = 0.19432034101933482
Epoch 4 ---- dif = 0.00023721153257838523
Epoch 5 ---- dif = 1.668065550575193e-06
Epoch 6 ---- dif = 2.6944860564981354e-08
Epoch 7 ---- dif = 4.782521045854082e-10
Epoch 8 ---- dif = 8.554934538551606e-12
Epoch 9 ---- dif = 2.2737367544323206e-13
